In [1]:
!pip install beautifulsoup4
!python.exe -m pip install --upgrade pip

In [2]:
import requests,bs4,os,json,time

## download html

In [3]:
url = 'https://www.nogizaka46.com/s/n46/diary/MEMBER/list?ima=0107&ct=48006/s/n46/diary/detail/100509?ima=2301&cd=MEMBER'
# html = requests.get(url)
# print('網頁下載中')
# html.raise_for_status()
# print('網頁下載成功')

## create Nogizaka46 directory

In [4]:
deskDir = 'Nogizaka46'
if os.path.exists(deskDir) == False:
    os.mkdir(deskDir)

## create member list

In [5]:
fn = 'member_list.json'

with open(fn,'r',encoding='utf-8') as fnobj:
    member_list = json.load(fnobj)
print(member_list)

['秋元真夏', '伊藤理々杏', '岩本蓮加', '梅澤美波', '遠藤さくら', '賀喜遥香', '掛橋沙耶香', '金川紗耶', '北川悠理', '久保史緒里', '黒見明香', '齋藤飛鳥', '阪口珠美', '佐藤楓', '佐藤璃果', '柴田柚菜', '鈴木絢音', '清宮レイ', '田村真佑', '筒井あやめ', '中村麗乃', '早川聖来', '林瑠奈', '樋口日奈', '松尾美佑', '向井 葉月', '矢久保美緒', '山崎怜奈', '山下美月', '弓木奈於', '吉田綾乃クリスティー', '与田祐希', '和田まあや', '五百城茉央', '池田瑛紗', '一ノ瀬美空', '井上和', '岡本姫奈', '小川彩', '奥田いろは', '川﨑桜', '菅原咲月', '冨里奈央', '中西アルノ']


In [6]:
# for i in range(len(member_list)):
#     folderpath = './Nogizaka46' + '/' + member_list[i]
#     if os.path.isdir(folderpath) == False:
#         path = os.path.join('./Nogizaka46', member_list[i]) 
#         os.mkdir(path)

## find blog links on current page

In [7]:
# main_objSoup = bs4.BeautifulSoup(html.text,'lxml')
# blogData = main_objSoup.find_all('a',class_='bl--card js-pos a--op hv--thumb')

In [8]:
# print(len(blogData))

In [9]:
# for i in range(len(bloglinks)):
#     print(bloglinks[i]['href'])

In [10]:
# print(url[0:26]+bloglinks[0]['href'])

In [11]:
# print(blogData[1])

In [12]:
def get_next_page(current_objSoup):
    global current_page,download_complete
    
    nextPages = current_objSoup.find_all('ul',class_ = "pager")
    nextpage = nextPages[0].find_all('li',class_ = 'coun')
#     print(len(nextpage))
#     if current_page == 3:
#         download_complete = True
#         return ''
    for i in range(len(nextpage)):
        if nextpage[i].text == str(current_page+1):
            current_page += 1
            targetpage = nextpage[i]
            link = targetpage.find('a')['href']
            return url[:26] + link
    #it is end
    download_complete = True
    return ''

In [13]:
def download_pictures(blog_url,blog_index):
    global current_new,download_complete,current_page,member_list,author_member
    #url = https://www.nogizaka46.com/s/n46/diary/detail/100577?ima=2936&cd=MEMBER
    html = requests.get(blog_url)
    time.sleep(1)
    objSoup = bs4.BeautifulSoup(html.text,'lxml')
    
    imgTag = objSoup.select('img')
#     print(len(imgTag))
    author_member = objSoup.find('p',class_ = 'bd--prof__name f--head').text
    
    blog_title = objSoup.find('h1', class_ = 'bd--hd__ttl f--head a--tx js-tdi').text
    if blog_title is None:
        blog_title = ''

    dateTime = objSoup.find('p', class_ = "bd--hd__date a--tx js-tdi").text
    date = dateTime[:10]
    date = date.replace('.','')
    
    data = [dateTime,author_member,blog_title]
        
    if data == current_new: #we can stop here
        download_complete = True
    else:
        #blog_title may not a good directory name
        blog_title = blog_title[:150]
        special_char = r'/\:*?"<>|'
        for i in special_char:
            blog_title = blog_title.replace(i,'')

        dir_path_member = './Nogizaka46/' + author_member
        
        if author_member[-5:] == '期生リレー':
            member = blog_title.split()[-1]
            if not member in member_list:
                member = ''.join(blog_title.split()[-2:])
                if not member in member_list:
                    print('成員名字找錯')
            dir_path = './Nogizaka46/' + author_member + '/' + member
            if os.path.exists(dir_path) == False:
                os.mkdir(dir_path)
                
            dir_path = './Nogizaka46/' + author_member + '/' + member + '/' + date + ' ' + blog_title
        else:
            dir_path = './Nogizaka46/' + author_member + '/' + date + ' ' + blog_title

        
        #the space at the end of blog_title should be remove
        dir_path = dir_path.rstrip(' .')
        
        if os.path.exists(dir_path_member) == False:
            os.mkdir(dir_path_member)
        if os.path.exists(dir_path) == False:
            os.mkdir(dir_path)
            if len(imgTag) > 0 :
                for i in range(len(imgTag)):
                    imgUrl = imgTag[i].get('src')

                    try:
                        is_effective_url = (not imgUrl is None) and ((imgUrl[-4:] in ['.jpg','.tif','.png']) or (imgUrl[-5:] in ['.jpeg']))
                        if is_effective_url: #we may catch something we don't want(no 'src') 
                            finUrl = url[:26] + imgUrl
                            
                            picture = requests.get(finUrl)
                            time.sleep(0.5)
                            picture.raise_for_status()
            #                 print('%s 圖片下載成功' %finUrl)

                            pictFile = open(os.path.join(dir_path,str(i).zfill(2) + (imgUrl[-4:] if imgUrl[-4:] in ['.jpg','.tif','.png'] else imgUrl[-5:])),'wb')
                            for diskStorage in picture.iter_content(1024*1024):
                                pictFile.write(diskStorage)
                            pictFile.close()
                    except requests.exceptions.HTTPError:
                        print(current_page,blog_index+1,'找不到網頁! 照片可能被鎖了',author_member,blog_title)
                        
                    except:
                        print(current_page,blog_index+1,'新的未知錯誤出現! 可能是異常個案',author_member,blog_title)
        else:
            print('看到重複的部落格了')
            


In [14]:
def update_current_new(current_new_url):
    html = requests.get(current_new_url)
    time.sleep(1)
    objSoup = bs4.BeautifulSoup(html.text,'lxml')
    
    author_member = objSoup.find('p',class_ = 'bd--prof__name f--head').text
    blog_title = objSoup.find('h1', class_ = 'bd--hd__ttl f--head a--tx js-tdi').text
    if blog_title is None:
        blog_title = ''
    dateTime = objSoup.find('p', class_ = "bd--hd__date a--tx js-tdi").text
    
    current_new_data = [dateTime,author_member,blog_title]
    
    fn = 'current_new.json'
    with open(fn,'w',encoding='utf-8') as obj:
        json.dump(current_new_data,obj,ensure_ascii=False)

In [15]:
def download_blogs(current_objSoup):
    global current_new,current_page,download_complete
    blogData = current_objSoup.find_all('a',class_='bl--card js-pos a--op hv--thumb')
    
    for i in range(len(blogData)):
        link = blogData[i]['href']
        blog_url = url[:26] + link
        
        if not download_complete:
            download_pictures(blog_url,i)
        
        if current_page == 1 and i == 0: #always update first
            update_current_new(blog_url)

## set some parameters

In [16]:
current_page = 1
current_url = url

download_complete = False

## get current new

In [17]:
fn = 'current_new.json'

try:
    with open(fn,'r',encoding='utf-8') as fnobj:
        current_new = json.load(fnobj)

except Exception: #the 1st run of this code
    current_new = ''

## main loop

In [18]:
while not download_complete:
    print(current_page)
    html = requests.get(current_url)
    current_objSoup = bs4.BeautifulSoup(html.text,'lxml')
    try:
        download_blogs(current_objSoup)
    except FileExistsError as e:
        print('檔案已存在! 載到重複的部落格')
        break
    except FileNotFoundError as e:
        print('檔案找不到! 可能部落格名稱太長或路徑有錯')
        break
    except NotADirectoryError as e:
        print('目錄名稱無效! 可能資料夾命名不合法')
        break
    except Exception as e:
        print( e,'可能為異常個案')
    current_url = get_next_page(current_objSoup)

1
2
3
4
5
6
7
8
9
10
11
